In [21]:
import numpy as np

def perturb_with_gaussian_noise_3(center1, center2, center3, std_x, std_y, std_z):    

    # Unpack the input coordinates
    a, b, c = center1
    d, e, f = center2
    x, y, z = center3

    # Generate random perturbations from Gaussian distributions
    perturb_a = np.random.normal(loc=0, scale=std_x)
    perturb_b = np.random.normal(loc=0, scale=std_y)
    perturb_c = np.random.normal(loc=0, scale=std_z)
    perturb_d = np.random.normal(loc=0, scale=std_x)
    perturb_e = np.random.normal(loc=0, scale=std_y)
    perturb_f = np.random.normal(loc=0, scale=std_z)
    perturb_x = np.random.normal(loc=0, scale=std_x)
    perturb_y = np.random.normal(loc=0, scale=std_y)
    perturb_z = np.random.normal(loc=0, scale=std_z)

    # Apply perturbations to the coordinates
    a_perturbed = a + perturb_a
    b_perturbed = b + perturb_b
    c_perturbed = c + perturb_c
    d_perturbed = d + perturb_d
    e_perturbed = e + perturb_e
    f_perturbed = f + perturb_f
    x_perturbed = x + perturb_x
    y_perturbed = y + perturb_y
    z_perturbed = z + perturb_z

    # Return perturbed coordinates as a tuple
    return ([a_perturbed, b_perturbed, c_perturbed], [d_perturbed, e_perturbed, f_perturbed], [x_perturbed, y_perturbed, z_perturbed])

In [6]:
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor_coverage_3(nrrd_file, center1, radii1, center2, radii2, center3, radii3):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()

    image = reader.GetOutput()

    # Create the 1st ellipsoid source
    sphere1 = vtk.vtkImageEllipsoidSource()
    sphere1.SetOutputScalarTypeToShort()
    sphere1.SetCenter(center1)
    Spacing = image.GetSpacing()
    sphere1.SetRadius(radii1[0] / Spacing[0], radii1[1] / Spacing[1], radii1[2] / Spacing[2])

    # Create the 2nd ellipsoid source
    sphere2 = vtk.vtkImageEllipsoidSource()
    sphere2.SetOutputScalarTypeToShort()
    sphere2.SetCenter(center2)
    Spacing = image.GetSpacing()
    sphere2.SetRadius(radii2[0] / Spacing[0], radii2[1] / Spacing[1], radii2[2] / Spacing[2])

    # Create the 3rd ellipsoid source
    sphere3 = vtk.vtkImageEllipsoidSource()
    sphere3.SetOutputScalarTypeToShort()
    sphere3.SetCenter(center3)
    Spacing = image.GetSpacing()
    sphere3.SetRadius(radii3[0] / Spacing[0], radii3[1] / Spacing[1], radii3[2] / Spacing[2])

    # Create the dimensions of the file
    size_image = image.GetDimensions()
    sphere1.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere1.Update()
    sphere2.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere2.Update()
    sphere3.SetWholeExtent(0, size_image[0] - 1, 0, size_image[1] - 1, 0, size_image[2] - 1)
    sphere3.Update()

    # Align the 1st iceball and tumor images
    sp1 = sphere1.GetOutput()
    sp1.SetOrigin(image.GetOrigin())
    
    # Align the 2nd iceball and tumor images
    sp2 = sphere2.GetOutput()
    sp2.SetOrigin(image.GetOrigin())
    
    # Align the 3rd iceball and tumor images
    sp3 = sphere3.GetOutput()
    sp3.SetOrigin(image.GetOrigin())

    # Logic AND operation between 1st iceball and tumor
    logic1 = vtk.vtkImageLogic()
    logic1.SetInput1Data(image)
    logic1.SetInput2Data(sp1)
    logic1.SetOperationToAnd()
    logic1.SetOutputTrueValue(1)
    logic1.Update()
    
    # Logic AND operation between 2nd iceball and tumor
    logic2 = vtk.vtkImageLogic()
    logic2.SetInput1Data(image)
    logic2.SetInput2Data(sp2)
    logic2.SetOperationToAnd()
    logic2.SetOutputTrueValue(1)
    logic2.Update()
    
    # Logic AND operation between 3rd iceball and tumor
    logic3 = vtk.vtkImageLogic()
    logic3.SetInput1Data(image)
    logic3.SetInput2Data(sp3)
    logic3.SetOperationToAnd()
    logic3.SetOutputTrueValue(1)
    logic3.Update()

    # Logic OR between (1st iceball and tumor) AND (2nd iceball and tumor)
    logic12 = vtk.vtkImageLogic()
    logic12.SetInput1Data(logic1.GetOutput())
    logic12.SetInput2Data(logic2.GetOutput())
    logic12.SetOperationToOr()
    logic12.SetOutputTrueValue(1)
    logic12.Update()

    # Logic OR between ((1st iceball and tumor) AND (2nd iceball and tumor)) AND (3rd iceball and tumor)
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(logic12.GetOutput())
    logic.SetInput2Data(logic3.GetOutput())
    logic.SetOperationToOr()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [10]:
# Calculate total size of the tumor
import vtk
import nrrd
import numpy
from vtk.util.numpy_support import vtk_to_numpy

def tumor(nrrd_file):
    # Read the NRRD file using vtkNrrdReader
    reader = vtk.vtkNrrdReader()
    reader.SetFileName(nrrd_file)
    reader.Update()
    image = reader.GetOutput()

    # Logic AND operation between tumor and tumor --> 100% of image
    logic = vtk.vtkImageLogic()
    logic.SetInput1Data(image)
    logic.SetInput2Data(image)
    logic.SetOperationToAnd()
    logic.SetOutputTrueValue(1)
    logic.Update()

    # Extract scalar data and calculate sum
    sc = logic.GetOutput().GetPointData().GetScalars()
    a = vtk_to_numpy(sc)
    return numpy.sum(a)

In [28]:
'''
Uses SD to find needle coordinates, finds tumor coverage, if tumor not entirely covered add a counter

Arguments: 
    - nrrd file
    - standard deviation in x, y, z directions
    - iceball center, tuple or array with 3 values
    - iceball radii, tuple or array with 3 values

Return: counter / simulation #
1. Loop through # times
2. Perturbed Coordinates
3. Tumor Ratio
4. If below 1.0
    Add to a counter
else keep on looping

Return percentage of tumor coverage

'''

# iterating Gaussian noise --> need start back at original coordinates
def perturbed_tumor_percentage_3(nrrd, simulate, std_x, std_y, std_z, center1, radii1, center2, radii2, center3, radii3):
    counter = 0
    for i in range (simulate):
        center_SD1, center_SD2, center_SD3 = perturb_with_gaussian_noise_3(center1, center2, center3, std_x, std_y, std_z)
        print(center_SD1)
        print(center_SD2)
        print(center_SD3)
        ratio = iceball_coverage_3(nrrd, center_SD1, radii1, center_SD2, radii2, center_SD3, radii3)
        print(ratio)
        if ratio < 1.0:
            counter += 1
    print (counter)
    print(f"Tumor Coverage Percentage: {counter / simulate * 100:.2f}%")

In [29]:
def iceball_coverage_3(nrrd_file, center1, radii1, center2, radii2, center3, radii3):
    numerator = tumor_coverage_3(nrrd_file, center1, radii1, center2, radii2, center3, radii3)
    denominator = tumor(nrrd_file)
    ratio = numerator / denominator
    return ratio

In [33]:
nrrd_file = '/Users/alvarocuervo/Desktop/teste-label.nrrd'
center1 = [123, 120, 22]
radii1 = [100.0, 100.0, 100.0]
center2 = [123, 120, 22]
radii2 = [10.0, 10.0, 10.0]
center3 = [123, 120, 22]
radii3 = [10.0, 10.0, 10.0]
sim = 50
std_x = 2.0
std_y = 2.0
std_z = 2.0

sum_result = perturbed_tumor_percentage_3(nrrd_file, sim, std_x, std_y, std_z, center1, radii1, center2, radii2, center3, radii3)
print(sum_result)

[125.29609458083648, 121.92762853733704, 18.477686863469245]
[122.94404900227042, 119.69012437804354, 22.06933950250883]
[122.2477258792105, 119.49046642381353, 22.160383123783696]
1.0
[122.12943698507073, 121.83294573467126, 18.39503702307803]
[121.80416759548976, 121.24760215346866, 22.832692316905018]
[124.29904859286762, 122.74631061250179, 22.33370206285201]
1.0
[125.3655681172152, 123.29982401414033, 24.796715877837425]
[122.44662864370721, 120.45651517424594, 21.7230226630851]
[122.99436063886746, 119.99300672599288, 18.203647662002446]
1.0
[125.87590071906313, 122.11100047200809, 25.189486545015754]
[122.15284381804008, 120.83588902168043, 21.476703732389375]
[123.45148909926708, 119.10935134735938, 19.413914173525917]
1.0
[123.03057191409613, 117.54379769988213, 22.29849332889997]
[124.95296883534856, 118.08629652871403, 21.88043941558458]
[121.58315923945831, 120.69985306738683, 22.543105460920383]
1.0
[121.39659424216656, 119.14099453302273, 22.67461919749849]
[123.376789460

1.0
[126.46895065053941, 118.68410150390234, 19.895616369037835]
[125.36201587252192, 119.16404669834448, 21.400333131186944]
[120.56478471911305, 120.27165400389667, 19.093749463592125]
1.0
[123.77159107950641, 119.5027198377657, 22.593146356586065]
[122.21768911613086, 121.68531233541695, 23.53259444622568]
[120.81249284097409, 118.56555106384336, 21.48228717031646]
1.0
0
Tumor Coverage Percentage: 0.00%
None
